In [1]:
import pandas as pd
import numpy as np
import sys
import re
import os
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image, ImageChops, ImageOps

from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler




Using TensorFlow backend.


Artwork information 

In [3]:
train_info = pd.read_csv('/Users/jenelleverrochi/Desktop/Capstone/train_info.csv')
all_data_info = pd.read_csv('/Users/jenelleverrochi/Desktop/Capstone/all_data_info.csv')

### Preprocessing

First I need to crop images so that they are all the same size.  I am going to crop them to size (256 X 256) and save them to a new directory.

In [4]:
input_directory = [
    '/Users/jenelleverrochi/Desktop/Capstone/trial300/'
]


In [5]:
def get_files(src_dir):
    f = []
    for (dirpath, dirnames, filenames) in os.walk(src_dir):
        f.extend(filenames)
        break
    return f

In [6]:
def makeThumb(f_in, f_out, size=(256,256), pad=False):
    
    image = Image.open(f_in)
    image.thumbnail(size, Image.ANTIALIAS)
    image_size = image.size

    if pad:
        thumb = image.crop( (0, 0, size[0], size[1]) )

        offset_x = max( (size[0] - image_size[0]) / 2, 0 )
        offset_y = max( (size[1] - image_size[1]) / 2, 0 )

        thumb = ImageChops.offset(thumb, offset_x, offset_y)

    else:
        thumb = ImageOps.fit(image, size, Image.ANTIALIAS)

    thumb.save(f_out)


source = "/Users/jenelleverrochi/Desktop/Capstone/trial300/31.jpg"

makeThumb(source, "/Users/jenelleverrochi/Desktop/Capstone/trial300out/31.jpg", pad=False)

Now I have my new cropped images saved.  They are all the same size, so now I can pull these into my notebook as a numpy array.

In [7]:
imaget = '/Users/jenelleverrochi/Desktop/Capstone/trial300out/'
def get_array(jpg):
    pic = Image.open(imaget+jpg)
    array = np.array(pic.getdata()).reshape(pic.size[0], pic.size[1],3)
    return array

get_array('80.jpg')

array([[[232, 216, 203],
        [245, 229, 216],
        [243, 227, 214],
        ..., 
        [232, 222, 213],
        [240, 230, 221],
        [250, 240, 231]],

       [[232, 216, 203],
        [245, 229, 216],
        [243, 227, 214],
        ..., 
        [232, 222, 213],
        [240, 230, 221],
        [250, 240, 231]],

       [[232, 216, 203],
        [245, 229, 216],
        [243, 227, 214],
        ..., 
        [232, 222, 213],
        [240, 230, 221],
        [250, 240, 231]],

       ..., 
       [[219, 211, 192],
        [242, 234, 215],
        [246, 237, 220],
        ..., 
        [242, 227, 224],
        [244, 229, 226],
        [247, 232, 229]],

       [[219, 211, 192],
        [242, 234, 215],
        [246, 237, 220],
        ..., 
        [242, 227, 224],
        [244, 229, 226],
        [247, 232, 229]],

       [[219, 211, 192],
        [242, 234, 215],
        [246, 237, 220],
        ..., 
        [242, 227, 224],
        [244, 229, 226],
        [247, 232,

In [8]:
test_array = []
for j in [imaget]:
    jpgs = get_files(j)
    for i in jpgs:
        test_array.append(get_array(i))
        


In [8]:
print(len(test_array[0]))
print(len(test_array[0][0]))
print(len(test_array[0][0][0]))

256
256
3


In [9]:
test_array[0].shape

(256, 256, 3)

### Model with KMeans

Now I have my test_array which contains the arrays for each of the 305 test images.
Each of the images is sized 256x256 and each pixel has 3 digits since the images are color (representing the proportion of red/green/blue). The array is 4-dimentional (305 x 256 x 256 x 3).  I will need to reshape this in order to model.


Since these are unlabeled, I am going to try to cluster with Kmeans first.

In [9]:
test_array_2d = np.reshape(test_array, (305,196608))


In [10]:
test_array_2d.shape

(305, 196608)

In [11]:
model = KMeans(n_clusters=6)
model.fit(test_array_2d)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [12]:
predicted = model.labels_

In [13]:
pd.DataFrame(predicted)[0].value_counts()

1    78
0    78
5    51
4    44
3    31
2    23
Name: 0, dtype: int64

In [14]:
predicted_df = pd.DataFrame(predicted, columns=['Prediction'])
predicted_df.head()

,Prediction
0,1
1,1
2,0
3,1
4,0


In [15]:
test_df = pd.DataFrame(test_array_2d)

In [16]:
test_df_kmeans = pd.concat([test_df, predicted_df], axis=1)

In [17]:
test_df_kmeans.head()

,0,1,2,3,4,5,6,7,8,9,...,196599,196600,196601,196602,196603,196604,196605,196606,196607,Prediction
0,106,70,44,108,72,46,108,72,48,104,...,118,81,52,120,83,57,120,82,61,1
1,54,55,60,57,64,70,48,63,68,35,...,33,42,41,31,40,39,29,38,37,1
2,45,25,0,47,27,2,49,29,4,52,...,76,34,10,81,37,12,77,33,8,0
3,4,24,33,8,28,37,9,29,38,7,...,112,92,94,116,97,99,125,106,108,1
4,15,27,17,22,33,25,33,44,36,46,...,130,121,116,134,125,120,142,133,128,0


In [18]:
jpgs_list = []
for i in jpgs:
    jpgs_list.append(i)
        
        
jpg_df = pd.DataFrame(jpgs_list, columns=['jpg'])
jpg_df.head()

,jpg
0,10.jpg
1,100.jpg
2,101.jpg
3,102.jpg
4,104.jpg


In [19]:
test_df_kmeans = pd.concat([test_df_kmeans, jpg_df], axis=1)
test_df_kmeans.head()

,0,1,2,3,4,5,6,7,8,9,...,196600,196601,196602,196603,196604,196605,196606,196607,Prediction,jpg
0,106,70,44,108,72,46,108,72,48,104,...,81,52,120,83,57,120,82,61,1,10.jpg
1,54,55,60,57,64,70,48,63,68,35,...,42,41,31,40,39,29,38,37,1,100.jpg
2,45,25,0,47,27,2,49,29,4,52,...,34,10,81,37,12,77,33,8,0,101.jpg
3,4,24,33,8,28,37,9,29,38,7,...,92,94,116,97,99,125,106,108,1,102.jpg
4,15,27,17,22,33,25,33,44,36,46,...,121,116,134,125,120,142,133,128,0,104.jpg


In [20]:
test_df_kmeans[test_df_kmeans['Prediction'] == 5]

,0,1,2,3,4,5,6,7,8,9,...,196600,196601,196602,196603,196604,196605,196606,196607,Prediction,jpg
21,240,226,215,235,221,210,230,216,205,231,...,194,147,216,194,145,217,193,145,5,125.jpg
24,196,188,169,196,188,169,196,188,169,195,...,185,167,195,184,166,194,183,165,5,130.jpg
39,142,144,131,133,135,122,121,123,110,116,...,163,158,150,159,154,147,156,151,5,150.jpg
50,227,156,110,229,158,112,232,161,115,232,...,184,148,198,182,146,196,180,144,5,163.jpg
52,175,151,117,176,152,118,176,152,118,175,...,97,75,127,105,84,126,105,84,5,165.jpg
55,188,205,235,187,204,234,187,204,234,188,...,206,201,207,204,199,206,203,198,5,170.jpg
58,255,255,255,255,255,255,255,255,255,255,...,255,255,255,255,255,255,255,255,5,175.jpg
60,162,124,113,154,116,105,152,114,103,160,...,133,114,183,135,112,184,134,109,5,178.jpg
62,180,202,200,173,195,193,168,189,190,170,...,49,46,50,46,43,47,43,40,5,18.jpg
67,209,207,212,209,207,212,208,206,211,209,...,214,212,214,213,211,213,212,210,5,186.jpg


In [24]:
#pictures here

These clusters look decent at best. I want to read in image labels and ultimately I am going to use the labels(genre) to make predictions using supervised learning.

### Pull in label

In [21]:
test_df = pd.concat([test_df, jpg_df], axis=1)
test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,196599,196600,196601,196602,196603,196604,196605,196606,196607,jpg
0,106,70,44,108,72,46,108,72,48,104,...,118,81,52,120,83,57,120,82,61,10.jpg
1,54,55,60,57,64,70,48,63,68,35,...,33,42,41,31,40,39,29,38,37,100.jpg
2,45,25,0,47,27,2,49,29,4,52,...,76,34,10,81,37,12,77,33,8,101.jpg
3,4,24,33,8,28,37,9,29,38,7,...,112,92,94,116,97,99,125,106,108,102.jpg
4,15,27,17,22,33,25,33,44,36,46,...,130,121,116,134,125,120,142,133,128,104.jpg


In [22]:
genre_df = train_info[['filename', 'genre']]
genre_df.columns = ['jpg', 'genre']

In [23]:
test_df_labels = pd.merge(test_df, genre_df, on='jpg', how='left')
test_df_labels.head()

,0,1,2,3,4,5,6,7,8,9,...,196600,196601,196602,196603,196604,196605,196606,196607,jpg,genre
0,106,70,44,108,72,46,108,72,48,104,...,81,52,120,83,57,120,82,61,10.jpg,religious painting
1,54,55,60,57,64,70,48,63,68,35,...,42,41,31,40,39,29,38,37,100.jpg,landscape
2,45,25,0,47,27,2,49,29,4,52,...,34,10,81,37,12,77,33,8,101.jpg,portrait
3,4,24,33,8,28,37,9,29,38,7,...,92,94,116,97,99,125,106,108,102.jpg,portrait
4,15,27,17,22,33,25,33,44,36,46,...,121,116,134,125,120,142,133,128,104.jpg,religious painting


In [24]:
test_df_labels.shape

(305, 196610)

There are 5 rows with null in Y columns - so we want to drop those

In [25]:
test_df_labels.dropna(inplace=True)

In [26]:
test_df_labels.shape

(300, 196610)

In [27]:
X = test_df_labels.iloc[:,:-2]
y = test_df_labels.iloc[:,-1]

In [28]:
y.shape

(300,)

In [29]:
y = pd.get_dummies(y)

In [30]:
y.shape

(300, 27)

In [31]:
X.shape

(300, 196608)

In [32]:
y.isnull().sum().sum()

0

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [34]:
X_train = X_train/255.
X_test = X_test/255.

In [35]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [36]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [37]:
X_train.shape

(225, 196608)

In [38]:
X_test.shape

(75, 196608)

In [39]:
y_train.shape

(225, 27)

In [40]:
y_test.shape

(75, 27)

### NN 1

In [48]:
model = Sequential()
model.add(Dense(500, input_dim=196608, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(50, activation='relu'))
model.add(Dense(27, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [49]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, verbose=1)

Train on 225 samples, validate on 75 samples
Epoch 1/5
225/225 [==============================] - 47s 208ms/step - loss: 12.0890 - acc: 0.1422 - val_loss: 12.8945 - val_acc: 0.2000
Epoch 2/5
225/225 [==============================] - 30s 135ms/step - loss: 13.1094 - acc: 0.1867 - val_loss: 12.8945 - val_acc: 0.2000
Epoch 3/5
225/225 [==============================] - 42s 186ms/step - loss: 13.1094 - acc: 0.1867 - val_loss: 12.8945 - val_acc: 0.2000
Epoch 4/5
225/225 [==============================] - 33s 145ms/step - loss: 13.1094 - acc: 0.1867 - val_loss: 12.8945 - val_acc: 0.2000
Epoch 5/5
225/225 [==============================] - 32s 143ms/step - loss: 13.1094 - acc: 0.1867 - val_loss: 12.8945 - val_acc: 0.2000


In [ ]:
model.score(X_test, y_test)

### NN with GridSearch

In [50]:
def model_funk():
    model = Sequential()
    model.add(Dense(500, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [51]:
ff_model = KerasClassifier(build_fn=model_funk, verbose=1)

params = {
    'epochs':[5,10]
}

gs = GridSearchCV(ff_model, param_grid=params)
gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

Epoch 1/5
150/150 [==============================] - 32s 214ms/step - loss: 10.8900 - acc: 0.1667
Epoch 2/5
150/150 [==============================] - 23s 155ms/step - loss: 12.4647 - acc: 0.2267
Epoch 3/5
150/150 [==============================] - 19s 124ms/step - loss: 12.4647 - acc: 0.2267
Epoch 4/5
150/150 [==============================] - 23s 152ms/step - loss: 12.4647 - acc: 0.2267
Epoch 5/5
150/150 [==============================] - 2s 12ms/step
Epoch 1/5
150/150 [==============================] - 33s 219ms/step - loss: 11.8524 - acc: 0.1000
Epoch 2/5
150/150 [==============================] - 22s 148ms/step - loss: 13.8616 - acc: 0.1400
Epoch 3/5
150/150 [==============================] - 24s 158ms/step - loss: 13.8616 - acc: 0.1400
Epoch 4/5
150/150 [==============================] - 20s 134ms/step - loss: 13.8616 - acc: 0.1400
Epoch 5/5
150/150 [==============================] - 2s 12ms/step
Epoch 1/5
150/150 [==============================] - 32s 213ms/step - loss: 11.3440 

In [53]:
gs.score(X_test, y_test)

75/75 [==============================] - 2s 23ms/step


0.10666666746139526

Model 2

In [54]:
pwd

'/Users/jenelleverrochi/GA/PROJECTS/CAPSTONE'

In [52]:
#this takes too long to run

### CNN Network

In [54]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(225, 196608)
(75, 196608)
(225, 27)
(75, 27)


In [56]:
X_train[0].shape

(196608,)

In [63]:
256 * 256 * 3

196608

In [59]:
X_train = X_train.reshape(X_train.shape[0], 256, 256, 3)
X_test = X_test.reshape(X_test.shape[0], 256, 256, 3)

In [64]:
model = Sequential()
model.add(Conv2D((15), (5,5), input_shape=(256, 256, 3), activation='relu'))
#15 magnifine glasses, 5X5 is shape of magnifine glass
model.add(MaxPool2D(2,2))#2 pixel jump so no overlap on the scan, can use gridsearch to find best pixel slide
model.add(Conv2D(30, (4,4), activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [66]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

Train on 225 samples, validate on 75 samples
Epoch 1/5
225/225 [==============================] - 40s 177ms/step - loss: 2.9947 - acc: 0.1556 - val_loss: 3.1158 - val_acc: 0.1200
Epoch 2/5
225/225 [==============================] - 38s 168ms/step - loss: 2.9310 - acc: 0.1733 - val_loss: 3.1010 - val_acc: 0.1067
Epoch 3/5
225/225 [==============================] - 38s 168ms/step - loss: 2.7697 - acc: 0.1600 - val_loss: 3.0556 - val_acc: 0.1067
Epoch 4/5
225/225 [==============================] - 38s 170ms/step - loss: 2.6220 - acc: 0.2311 - val_loss: 3.2006 - val_acc: 0.1200
Epoch 5/5
225/225 [==============================] - 42s 186ms/step - loss: 2.4955 - acc: 0.3244 - val_loss: 3.0621 - val_acc: 0.2267


In [69]:
model = Sequential()
model.add(Conv2D(15, kernel_size=(5,5), input_shape=(256,256,3), activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(30, kernel_size=(4,4), activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Conv2D(45, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D((2,2)))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(27, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5)

Train on 225 samples, validate on 75 samples
Epoch 1/5
225/225 [==============================] - 46s 203ms/step - loss: 3.1666 - acc: 0.1689 - val_loss: 3.0138 - val_acc: 0.2000
Epoch 2/5
225/225 [==============================] - 42s 185ms/step - loss: 3.0315 - acc: 0.1778 - val_loss: 3.0560 - val_acc: 0.1067
Epoch 3/5
225/225 [==============================] - 42s 187ms/step - loss: 2.9198 - acc: 0.1689 - val_loss: 3.0139 - val_acc: 0.1067
Epoch 4/5
225/225 [==============================] - 45s 201ms/step - loss: 2.9031 - acc: 0.1956 - val_loss: 3.0185 - val_acc: 0.1067
Epoch 5/5
225/225 [==============================] - 47s 210ms/step - loss: 2.8298 - acc: 0.1644 - val_loss: 3.0118 - val_acc: 0.1467


My models suck - I want to take a look at the images in each genre to make sure they are labeled properly.

In [70]:
test_df_labels.head()

,0,1,2,3,4,5,6,7,8,9,...,196600,196601,196602,196603,196604,196605,196606,196607,jpg,genre
0,106,70,44,108,72,46,108,72,48,104,...,81,52,120,83,57,120,82,61,10.jpg,religious painting
1,54,55,60,57,64,70,48,63,68,35,...,42,41,31,40,39,29,38,37,100.jpg,landscape
2,45,25,0,47,27,2,49,29,4,52,...,34,10,81,37,12,77,33,8,101.jpg,portrait
3,4,24,33,8,28,37,9,29,38,7,...,92,94,116,97,99,125,106,108,102.jpg,portrait
4,15,27,17,22,33,25,33,44,36,46,...,121,116,134,125,120,142,133,128,104.jpg,religious painting


In [71]:
test_df_labels.genre.value_counts()

portrait                    57
landscape                   43
genre painting              32
religious painting          25
abstract                    24
cityscape                   20
illustration                11
still life                   9
symbolic painting            9
flower painting              8
figurative                   7
mythological painting        7
sketch and study             6
animal painting              6
design                       6
marina                       6
self-portrait                4
interior                     4
nude painting (nu)           3
capriccio                    2
battle painting              2
literary painting            2
pastorale                    2
allegorical painting         2
miniature                    1
tessellation                 1
bird-and-flower painting     1
Name: genre, dtype: int64

In [85]:
# for i in get_files(imaget):
Image.open('/Users/jenelleverrochi/Desktop/Capstone/trial300out/5.jpg').show()
Image.open('/Users/jenelleverrochi/Desktop/Capstone/trial300out/6.jpg').show()


In [93]:
imaget

'/Users/jenelleverrochi/Desktop/Capstone/trial300out/'

In [96]:
# prints all paintings in 300out folder
filelist = get_files(imaget)
for i in filelist:    
    im=Image.open(imaget+i)
    im.show()


Print all images in genres: portrait, landscape, genre painting, religious painting, abstract, cityscape

In [101]:
#Portrait
portrait_df = test_df_labels[test_df_labels['genre'] == 'portrait']

portrait_jpg_lst= []
for j in portrait_df.jpg:
    portrait_jpg_lst.append(j)

    

for i in portrait_jpg_lst:    
    im=Image.open(imaget+i)
    im.show()

In [111]:
#Landscape
landscape_df = test_df_labels[test_df_labels['genre'] == 'landscape']

landscape_jpg_lst= []
for j in landscape_df.jpg:
    landscape_jpg_lst.append(j)

for i in landscape_jpg_lst:    
    im=Image.open(imaget+i)
    im.show()

In [113]:
#genre painting:
#a style of painting depicting scenes from ordinary life, 
#especially domestic situations. Genre painting is associated particularly with 
#17th-century Dutch and Flemish artists.

genre_painting_df = test_df_labels[test_df_labels['genre'] == 'genre painting']

genre_jpg_lst= []
for j in genre_painting_df.jpg:
    genre_jpg_lst.append(j)

for i in genre_jpg_lst:    
    im=Image.open(imaget+i)
    im.show()

In [53]:
#religious painting
religious_df = test_df_labels[test_df_labels['genre'] == 'religious painting']

religious_jpg_lst= []
for j in religious_df.jpg:
    religious_jpg_lst.append(j)

for i in religious_jpg_lst:    
    im=Image.open(imaget+i)
    im.show()

In [116]:
#abstract
abstract_df = test_df_labels[test_df_labels['genre'] == 'abstract']

abstract_jpg_lst= []
for j in abstract_df.jpg:
    abstract_jpg_lst.append(j)

for i in abstract_jpg_lst:    
    im=Image.open(imaget+i)
    im.show()

In [117]:
#cityscape
cityscape_df = test_df_labels[test_df_labels['genre'] == 'cityscape']

cityscape_jpg_lst= []
for j in cityscape_df.jpg:
    cityscape_jpg_lst.append(j)

for i in cityscape_jpg_lst:    
    im=Image.open(imaget+i)
    im.show()

After looking at each group of pictures, it is clear to me why my models are performing so poorly.  A lot of the images between categories are very similar and look as if they could be classified in several of the genres.  For example, there are many religious paintings that could also be considered portraits or landscapes.  The genre paintings are also very ambiguous.  These are mainly scenes from everyday life, but there is a wide range of styles.  A lot of paintings from this genre could fall into other genres as well, such as portraits, religious paintings, landscapes, etc.

I may change my strategy somewhat - I am going to pull in styles and artists and see if either of these features differentiate better.  These may be more telling/predictive.

### Pull in styles and artists

In [41]:
test_df_labels.head()

,0,1,2,3,4,5,6,7,8,9,...,196600,196601,196602,196603,196604,196605,196606,196607,jpg,genre
0,106,70,44,108,72,46,108,72,48,104,...,81,52,120,83,57,120,82,61,10.jpg,religious painting
1,54,55,60,57,64,70,48,63,68,35,...,42,41,31,40,39,29,38,37,100.jpg,landscape
2,45,25,0,47,27,2,49,29,4,52,...,34,10,81,37,12,77,33,8,101.jpg,portrait
3,4,24,33,8,28,37,9,29,38,7,...,92,94,116,97,99,125,106,108,102.jpg,portrait
4,15,27,17,22,33,25,33,44,36,46,...,121,116,134,125,120,142,133,128,104.jpg,religious painting


In [42]:
all_data_info.head()

,artist,date,genre,pixelsx,pixelsy,size_bytes,source,style,title,artist_group,in_train,new_filename
0,Barnett Newman,1955.0,abstract,15530.0,6911.0,9201912.0,wikiart,Color Field Painting,Uriel,train_only,True,102257.jpg
1,Barnett Newman,1950.0,abstract,14559.0,6866.0,8867532.0,wikiart,Color Field Painting,Vir Heroicus Sublimis,train_only,True,75232.jpg
2,kiri nichol,2013.0,NaN,9003.0,9004.0,1756681.0,NaN,Neoplasticism,NaN,test_only,False,32145.jpg
3,kiri nichol,2013.0,NaN,9003.0,9004.0,1942046.0,NaN,Neoplasticism,NaN,test_only,False,20304.jpg
4,kiri nichol,2013.0,NaN,9003.0,9004.0,1526212.0,NaN,Neoplasticism,NaN,test_only,False,836.jpg


In [46]:
artist_style_df = all_data_info[['new_filename', 'artist', 'style']]
artist_style_df.columns = ['jpg', 'artist', 'style']

In [48]:
artist_style_df.head()

,jpg,artist,style
0,102257.jpg,Barnett Newman,Color Field Painting
1,75232.jpg,Barnett Newman,Color Field Painting
2,32145.jpg,kiri nichol,Neoplasticism
3,20304.jpg,kiri nichol,Neoplasticism
4,836.jpg,kiri nichol,Neoplasticism


In [49]:
test_df_labels2 = pd.merge(test_df_labels, artist_style_df, on='jpg', how='left')
test_df_labels2.head()

,0,1,2,3,4,5,6,7,8,9,...,196602,196603,196604,196605,196606,196607,jpg,genre,artist,style
0,106,70,44,108,72,46,108,72,48,104,...,120,83,57,120,82,61,10.jpg,religious painting,Jacob Jordaens,Baroque
1,54,55,60,57,64,70,48,63,68,35,...,31,40,39,29,38,37,100.jpg,landscape,Karlovo,Post-Impressionism
2,45,25,0,47,27,2,49,29,4,52,...,81,37,12,77,33,8,101.jpg,portrait,Eugene Carriere,Symbolism
3,4,24,33,8,28,37,9,29,38,7,...,116,97,99,125,106,108,102.jpg,portrait,Leonor Fini,Magic Realism
4,15,27,17,22,33,25,33,44,36,46,...,134,125,120,142,133,128,104.jpg,religious painting,Peter Paul Rubens,Baroque


In [52]:
test_df_labels2['style'].value_counts()

Realism                         31
Romanticism                     30
Impressionism                   25
Art Nouveau (Modern)            21
Expressionism                   21
Post-Impressionism              16
Symbolism                       12
Surrealism                      10
Baroque                          9
Abstract Expressionism           8
Cubism                           8
Naïve Art (Primitivism)          7
Rococo                           7
Early Renaissance                6
Ukiyo-e                          6
New Realism                      5
Neo-Romanticism                  4
Northern Renaissance             4
High Renaissance                 4
Color Field Painting             4
Art Informel                     4
Op Art                           3
Mannerism (Late Renaissance)     3
Fauvism                          3
Post-Painterly Abstraction       2
Hard Edge Painting               2
Divisionism                      2
Pop Art                          2
Art Deco            

In [ ]:
#Realism
realism_df = test_df_labels2[test_df_labels2['style'] == 'realism']

cityscape_jpg_lst= []
for j in cityscape_df.jpg:
    cityscape_jpg_lst.append(j)

for i in cityscape_jpg_lst:    
    im=Image.open(imaget+i)
    im.show()